In [1]:
!pip install plotly

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import random
import math

from numpy import mean, std
from scipy import stats
import scipy as sp

from matplotlib import pyplot
import seaborn as sns

import plotly.express as px

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV,  cross_val_score, RepeatedStratifiedKFold, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import get_scorer, make_scorer, confusion_matrix, classification_report, recall_score, precision_score, accuracy_score, fbeta_score, roc_curve, roc_auc_score, f1_score, confusion_matrix, mean_squared_error, log_loss
from sklearn import svm, datasets
from sklearn.naive_bayes import BernoulliNB
from sklearn.datasets import make_classification
from sklearn.exceptions import ConvergenceWarning
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.exceptions import ConvergenceWarning
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import PCA


from warnings import simplefilter
import imblearn
from imblearn.over_sampling import SMOTE

from sklearn.base import clone
from sklearn.preprocessing import FunctionTransformer



simplefilter("ignore", category = ConvergenceWarning)

In [3]:
path = "/Users/yaeerk/Documents/NAYA/classification/"
jobs = 8

In [4]:
df = pd.read_csv(path + 'PROTECT_and_RISK_shared_DEGs_NOT NORM.csv', index_col = 0)

In [5]:
X = df.drop('Diagnosis', axis = 1)
y = df.Diagnosis
y = LabelEncoder().fit_transform(y)
numerical_cols = X.columns.to_list()

In [6]:
tab = "  "
testsizes = [0.08, 0.1, 0.15, 0.2]
#randomstates = [132, 400, 1440, 1600, 2500, 3333, 4567]
randomstates = [250, 650, 850, 1050, 1250, 1850, 2050]

In [7]:
#pca params
ncomponentss = [2, 3, 5, 10, 15, 25, 50, 100]

#cross validation
cv = 5

#scores we want
scores = ['f1', 'accuracy']

results = pd.DataFrame(columns = ["score", "test score", "train score", "test variance", "train variance", "test rmse", "train rmse", "test log_loss", "train log_loss", "test size", "random state", "estimator", "estimator params"])
results.head()

,score,test score,train score,test variance,train variance,test rmse,train rmse,test log_loss,train log_loss,test size,random state,estimator,estimator params


In [8]:
#random forest
max_depths = [10, 20, 30]
min_samples_leafs = [3, 10]

for testsize in testsizes:
  print(f"test size: {testsize}")
    
  for randomstate in randomstates:
    print(tab * 2 + f"random state: {randomstate}")
    
    for ncomponents in ncomponentss:
        print(tab * 3 + f"ncomponents: {ncomponents}")
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testsize, random_state = randomstate)                     
        
        smote = SMOTE(random_state = randomstate)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        robustscaler = RobustScaler(quantile_range = (1, 99))
        robustscaler.fit(X_train)
        X_train = robustscaler.transform(X_train)
        X_test  = robustscaler.transform(X_test)
        
        pca = PCA(n_components = ncomponents)        
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_test  = pca.transform(X_test)

        best_avg_scores = {score : [None] for score in scores}
        
        # Run Grid search for each classifier
        for max_depth in max_depths:
            for min_samples_leaf in min_samples_leafs:
                    
                rf = RandomForestClassifier(max_depth = max_depth, min_samples_leaf = min_samples_leaf, random_state = randomstate)
                cv_results = cross_validate(rf, X_train, y_train, cv = cv, scoring = scores, return_train_score = True, n_jobs = jobs)
 
                for score in scores:
                    avg_score_test = np.mean(cv_results['test_' + score])
                    var_score_test = np.var(cv_results['test_' + score])
                    avg_score_train = np.mean(cv_results['train_' + score])
                    var_score_train = np.var(cv_results['train_' + score])

                    if(best_avg_scores[score][0] is None or avg_score_test > best_avg_scores[score][0]):
                        best_avg_scores[score] = [avg_score_test, var_score_test, avg_score_train, var_score_train, max_depth, min_samples_leaf]

        for score in scores: 

            print(tab * 4 + str(score))
            print(tab * 5 + f"CV score: {best_avg_scores[score][0]} using:" + ','.join([str(p) for p in best_avg_scores[score][4:]]))
            print(tab * 6 + f"train score: {best_avg_scores[score][2]} with variance: {best_avg_scores[score][3]}")
            print(tab * 6 + f"test  score: {best_avg_scores[score][0]} with variance: {best_avg_scores[score][1]}")

            rf = RandomForestClassifier(max_depth = best_avg_scores[score][4], min_samples_leaf = best_avg_scores[score][5], random_state = randomstate)

            rf.fit(X_train, y_train)            
            y_train_pred, y_test_pred = rf.predict(X_train), rf.predict(X_test)                          
            rmse_train, rmse_test = math.sqrt(mean_squared_error(y_train, y_train_pred)), math.sqrt(mean_squared_error(y_test, y_test_pred))                    
            log_loss_train, log_loss_test = log_loss(y_train, y_train_pred), log_loss(y_test, y_test_pred)        

            score_train, score_test = get_scorer(score)(rf, X_train, y_train), get_scorer(score)(rf, X_test, y_test)

            print(tab * 5 + f"Refitted train score: {score_train},  RMSE: {rmse_train}, Log-Loss:{log_loss_train}")
            print(tab * 5 + f"Refitted test  score: {score_test},  RMSE: {rmse_test}, Log-Loss:{log_loss_test}")

            n = len(results)
            results.at[n, 'score'] = score
            results.at[n, 'test score'] = best_avg_scores[score][0]
            results.at[n, 'train score'] = best_avg_scores[score][2]
            results.at[n, 'test variance'] = best_avg_scores[score][1]
            results.at[n, 'train variance'] = best_avg_scores[score][3]
            results.at[n, 'test rmse'] = rmse_test
            results.at[n, 'train rmse'] = rmse_train
            results.at[n, 'test log_loss'] = log_loss_test
            results.at[n, 'train log_loss'] = log_loss_train
            results.at[n, 'test size'] = testsize
            results.at[n, 'random state'] = randomstate
            results.at[n, 'estimator'] = "Smote/pca/RandomForestClassifier"
            results.at[n, 'estimator params'] = ','.join([str(p) for p in best_avg_scores[score][4:]])

test size: 0.08
    random state: 250
      ncomponents: 2
        f1
          CV score: 0.9447023000880703 using:10,10
            train score: 0.958075887687827 with variance: 1.2175329337044011e-05
            test  score: 0.9447023000880703 with variance: 0.0007971853957678977
          Refitted train score: 0.956140350877193,  RMSE: 0.20896918976428644, Log-Loss:1.5082592233080474
          Refitted test  score: 0.975609756097561,  RMSE: 0.19611613513818404, Log-Loss:1.3284452304746504
        accuracy
          CV score: 0.9453655040611562 using:10,10
            train score: 0.958513125176814 with variance: 1.0283989006437225e-05
            test  score: 0.9453655040611562 with variance: 0.0007246392030039381
          Refitted train score: 0.9563318777292577,  RMSE: 0.20896918976428644, Log-Loss:1.5082592233080474
          Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.3284452304746504
      ncomponents: 3
        f1
          CV score: 0.949

          Refitted train score: 0.9696969696969697,  RMSE: 0.17407765595569785, Log-Loss:1.0466399721067772
          Refitted test  score: 0.8846153846153846,  RMSE: 0.3396831102433787, Log-Loss:3.9852434301820026
      ncomponents: 5
        f1
          CV score: 0.9610996888416243 using:10,10
            train score: 0.9743468044222814 with variance: 5.260489343829373e-06
            test  score: 0.9610996888416243 with variance: 0.0008989648943024748
          Refitted train score: 0.975929978118162,  RMSE: 0.1543033499620919, Log-Loss:0.8223570111175509
          Refitted test  score: 0.8750000000000001,  RMSE: 0.3922322702763681, Log-Loss:5.313657906909336
        accuracy
          CV score: 0.9611500701262272 using:10,10
            train score: 0.9745696916428622 with variance: 4.558690454580273e-06
            test  score: 0.9611500701262272 with variance: 0.0009026612558987665
          Refitted train score: 0.9761904761904762,  RMSE: 0.1543033499620919, Log-Loss:0.82235701

        f1
          CV score: 0.9649623868417697 using:10,3
            train score: 0.9840792586273743 with variance: 2.2534443099724773e-05
            test  score: 0.9649623868417697 with variance: 0.0002594041719415763
          Refitted train score: 0.9868421052631579,  RMSE: 0.11396057645963795, Log-Loss:0.4485555375962435
          Refitted test  score: 0.972972972972973,  RMSE: 0.19611613513818404, Log-Loss:1.32844523047465
        accuracy
          CV score: 0.9654277699859748 using:10,3
            train score: 0.9843052808906467 with variance: 2.1698186334208362e-05
            test  score: 0.9654277699859748 with variance: 0.00024906667750743507
          Refitted train score: 0.987012987012987,  RMSE: 0.11396057645963795, Log-Loss:0.4485555375962435
          Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.32844523047465
      ncomponents: 15
        f1
          CV score: 0.9669033385137927 using:10,3
            train score: 0.992921381

          Refitted train score: 0.9935064935064936,  RMSE: 0.08058229640253803, Log-Loss:0.22427776879812228
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
      ncomponents: 25
        f1
          CV score: 0.9693091854382179 using:10,3
            train score: 0.994011356875723 with variance: 1.1945012842545907e-06
            test  score: 0.9693091854382179 with variance: 0.0003058184469281193
          Refitted train score: 0.9956521739130434,  RMSE: 0.0657951694959769, Log-Loss:0.14951851253208187
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
        accuracy
          CV score: 0.9697522206638615 using:10,3
            train score: 0.9940481945359995 with variance: 1.1624531906860996e-06
            test  score: 0.9697522206638615 with variance: 0.0002966207179780378
          Refitted train score: 0.9956709956709957,  RMSE: 0.0657951694959769, Log-Loss:0.14951851253208187
          Refitted test  score: 1.0,  RM

          Refitted train score: 0.9933481152993348,  RMSE: 0.08128917219051073, Log-Loss:0.2282297999663711
          Refitted test  score: 0.9777777777777777,  RMSE: 0.19611613513818404, Log-Loss:1.32844523047465
        accuracy
          CV score: 0.966984126984127 using:20,3
            train score: 0.9922910422910423 with variance: 4.244940218058094e-06
            test  score: 0.966984126984127 with variance: 9.554282374795128e-05
          Refitted train score: 0.9933920704845814,  RMSE: 0.08128917219051073, Log-Loss:0.2282297999663711
          Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.32844523047465
      ncomponents: 100
        f1
          CV score: 0.9689112755324103 using:20,3
            train score: 0.9983440634369817 with variance: 4.88050153304447e-06
            test  score: 0.9689112755324103 with variance: 0.0001570750113068241
          Refitted train score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
          Refitted t

...

        f1
          CV score: 0.9733158737087152 using:10,3
            train score: 0.9993808049535604 with variance: 1.5336100221415305e-06
            test  score: 0.9733158737087152 with variance: 0.00033310040708851835
          Refitted train score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
          Refitted test  score: 0.9787234042553191,  RMSE: 0.17541160386140583, Log-Loss:1.062743882880794
        accuracy
          CV score: 0.9728086419753087 using:10,3
            train score: 0.9993808049535604 with variance: 1.5336100221415305e-06
            test  score: 0.9728086419753087 with variance: 0.000389273738759336
          Refitted train score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
          Refitted test  score: 0.9692307692307692,  RMSE: 0.17541160386140583, Log-Loss:1.062743882880794
    random state: 850
      ncomponents: 2
        f1
          CV score: 0.9396708620337313 using:10,3
            train score: 0.9659462339196644 with variance: 1.7825959

          Refitted train score: 0.9679802955665024,  RMSE: 0.17894050529015934, Log-Loss:1.1059292894668964
          Refitted test  score: 0.9538461538461539,  RMSE: 0.21483446221182986, Log-Loss:1.594109673571728
      ncomponents: 3
        f1
          CV score: 0.9579406380461233 using:10,10
            train score: 0.9621864749790632 with variance: 2.0408204211293293e-05
            test  score: 0.9579406380461233 with variance: 0.0004048035703642949
          Refitted train score: 0.9627791563275433,  RMSE: 0.1922129361096224, Log-Loss:1.2760749840104475
          Refitted test  score: 0.967032967032967,  RMSE: 0.21483446221182986, Log-Loss:1.594109673571728
        accuracy
          CV score: 0.9581451370069256 using:10,10
            train score: 0.9624368471035136 with variance: 2.0573700249547107e-05
            test  score: 0.9581451370069256 with variance: 0.000400651334961943
          Refitted train score: 0.9630541871921182,  RMSE: 0.1922129361096224, Log-Loss:1.276074

        f1
          CV score: 0.9668085037325543 using:10,3
            train score: 0.9767775797129712 with variance: 1.81775691530001e-05
            test  score: 0.9668085037325543 with variance: 0.0005389093076146659
          Refitted train score: 0.9742268041237113,  RMSE: 0.15971914124998499, Log-Loss:0.8810973539321372
          Refitted test  score: 0.9814814814814815,  RMSE: 0.17541160386140583, Log-Loss:1.0627561843797202
        accuracy
          CV score: 0.9668938656280428 using:10,3
            train score: 0.9770415742455384 with variance: 1.7919405828635134e-05
            test  score: 0.9668938656280428 with variance: 0.0005499148227754342
          Refitted train score: 0.9744897959183674,  RMSE: 0.15971914124998499, Log-Loss:0.8810973539321372
          Refitted test  score: 0.9692307692307692,  RMSE: 0.17541160386140583, Log-Loss:1.0627561843797202
      ncomponents: 10
        f1
          CV score: 0.9668717948717948 using:10,3
            train score: 0.979375

          Refitted train score: 0.9902439024390244,  RMSE: 0.09877295966495896, Log-Loss:0.336963672145471
          Refitted test  score: 0.9538461538461539,  RMSE: 0.21483446221182986, Log-Loss:1.5941096735717277
      ncomponents: 15
        f1
          CV score: 0.9700593842787935 using:10,3
            train score: 0.9932420837825239 with variance: 5.338948067526237e-06
            test  score: 0.9700593842787935 with variance: 0.00040405379967241246
          Refitted train score: 0.9926289926289926,  RMSE: 0.08553989227683016, Log-Loss:0.2527227541091035
          Refitted test  score: 0.9545454545454546,  RMSE: 0.2480694691784169, Log-Loss:2.125487765761587
        accuracy
          CV score: 0.9707317073170731 using:10,3
            train score: 0.9932926829268294 with variance: 5.205234979179069e-06
            test  score: 0.9707317073170731 with variance: 0.00039262343842950704
          Refitted train score: 0.9926829268292683,  RMSE: 0.08553989227683016, Log-Loss:0.2527

          Refitted train score: 0.9921875,  RMSE: 0.08838834764831845, Log-Loss:0.2698362728702151
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626415e-16
      ncomponents: 50
        f1
          CV score: 0.963782666428196 using:10,3
            train score: 0.995422693667631 with variance: 6.829393927589955e-06
            test  score: 0.963782666428196 with variance: 0.00033693559467695645
          Refitted train score: 0.9973890339425587,  RMSE: 0.05103103630798288, Log-Loss:0.08994473019508091
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626415e-16
        accuracy
          CV score: 0.963568010936432 using:10,3
            train score: 0.9954439697110707 with variance: 6.774059193644579e-06
            test  score: 0.963568010936432 with variance: 0.00036290938618527963
          Refitted train score: 0.9973958333333334,  RMSE: 0.05103103630798288, Log-Loss:0.08994473019508091
          Refitted test  score: 1.0,  RMSE: 0.0, Log

In [9]:
results.tail()

,score,test score,train score,test variance,train variance,test rmse,train rmse,test log_loss,train log_loss,test size,random state,estimator,estimator params
443,accuracy,0.9662,0.993492,0.000241458,1.26911e-05,0,0.0883883,9.99201e-16,0.269836,0.2,2050,Smote/pca/RandomForestClassifier,"10,3"
444,f1,0.963783,0.995423,0.000336936,6.82939e-06,0,0.051031,9.99201e-16,0.0899447,0.2,2050,Smote/pca/RandomForestClassifier,"10,3"
445,accuracy,0.963568,0.995444,0.000362909,6.77406e-06,0,0.051031,9.99201e-16,0.0899447,0.2,2050,Smote/pca/RandomForestClassifier,"10,3"
446,f1,0.960736,0.999349,0.000487339,1.69763e-06,0,0,9.99201e-16,9.99201e-16,0.2,2050,Smote/pca/RandomForestClassifier,"10,3"
447,accuracy,0.960971,0.999349,0.000470498,1.69763e-06,0,0,9.99201e-16,9.99201e-16,0.2,2050,Smote/pca/RandomForestClassifier,"10,3"


In [10]:
#AdaBoostRegressor
for testsize in testsizes:
  print(f"test size: {testsize}")
    
  for randomstate in randomstates:
    print(tab * 2 + f"random state: {randomstate}")
        
    for ncomponents in ncomponentss:
        print(tab * 3 + f"ncomponents: {ncomponents}")
            
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testsize, random_state = randomstate)
                     
        smote = SMOTE(random_state = randomstate)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        robustscaler = RobustScaler(quantile_range = (1, 99))
        robustscaler.fit(X_train)
                                    
        X_train = robustscaler.transform(X_train)
        X_test  = robustscaler.transform(X_test)

        pca = PCA(n_components = ncomponents)        
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_test  = pca.transform(X_test)

        best_avg_scores = {score : [None] for score in scores}
        
        ab = AdaBoostClassifier(random_state = randomstate)
        cv_results = cross_validate(ab, X_train, y_train, cv = cv, scoring = scores, return_train_score = True, n_jobs = jobs)

        for score in scores:
            avg_score_test = np.mean(cv_results['test_' + score])
            var_score_test = np.var(cv_results['test_' + score])
            avg_score_train = np.mean(cv_results['train_' + score])
            var_score_train = np.var(cv_results['train_' + score])

            if(best_avg_scores[score][0] is None or avg_score_test > best_avg_scores[score][0]):
                best_avg_scores[score] = [avg_score_test, var_score_test, avg_score_train, var_score_train, ncomponents]


        for score in scores: 

            print(tab * 4 + str(score))
            print(tab * 5 + f"CV score: {best_avg_scores[score][0]} using:" + ','.join([str(p) for p in best_avg_scores[score][4:]]))
            print(tab * 6 + f"train score: {best_avg_scores[score][2]} with variance: {best_avg_scores[score][3]}")
            print(tab * 6 + f"test  score: {best_avg_scores[score][0]} with variance: {best_avg_scores[score][1]}")

            ab = LogisticRegression(C = best_avg_scores[score][4], solver = 'liblinear', max_iter = 200, class_weight = 'balanced', random_state = randomstate)

            ab.fit(X_train, y_train)            
            y_train_pred, y_test_pred = ab.predict(X_train), ab.predict(X_test)                          
            rmse_train, rmse_test = math.sqrt(mean_squared_error(y_train, y_train_pred)), math.sqrt(mean_squared_error(y_test, y_test_pred))                    
            log_loss_train, log_loss_test = log_loss(y_train, y_train_pred), log_loss(y_test, y_test_pred)        

            score_train, score_test = get_scorer(score)(ab, X_train, y_train), get_scorer(score)(ab, X_test, y_test)

            print(tab * 5 + f"Refitted train score: {score_train},  RMSE: {rmse_train}, Log-Loss:{log_loss_train}")
            print(tab * 5 + f"Refitted test  score: {score_test},  RMSE: {rmse_test}, Log-Loss:{log_loss_test}") 

            n = len(results)
            results.at[n, 'score'] = score
            results.at[n, 'test score'] = best_avg_scores[score][0]
            results.at[n, 'train score'] = best_avg_scores[score][2]
            results.at[n, 'test variance'] = best_avg_scores[score][1]
            results.at[n, 'train variance'] = best_avg_scores[score][3]
            results.at[n, 'test rmse'] = rmse_test
            results.at[n, 'train rmse'] = rmse_train
            results.at[n, 'test log_loss'] = log_loss_test
            results.at[n, 'train log_loss'] = log_loss_train
            results.at[n, 'test size'] = testsize
            results.at[n, 'random state'] = randomstate
            results.at[n, 'estimator'] = "Smote/pca/AdaBoostRegressor"
            results.at[n, 'estimator params'] = ','.join([str(p) for p in best_avg_scores[score][4:]])

test size: 0.08
    random state: 250
      ncomponents: 2
        f1
          CV score: 0.9317709592443 using:2
            train score: 0.9917230708730054 with variance: 2.4605176642625934e-05
            test  score: 0.9317709592443 with variance: 0.00037920192970407687
          Refitted train score: 0.9490022172949001,  RMSE: 0.22409449036367163, Log-Loss:1.7344940040663481
          Refitted test  score: 0.975609756097561,  RMSE: 0.19611613513818404, Log-Loss:1.3284452304746504
        accuracy
          CV score: 0.9322981366459627 using:2
            train score: 0.9918062566072571 with variance: 2.390474923258006e-05
            test  score: 0.9322981366459627 with variance: 0.00035841378754609206
          Refitted train score: 0.9497816593886463,  RMSE: 0.22409449036367163, Log-Loss:1.7344940040663481
          Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.3284452304746504
      ncomponents: 3
        f1
          CV score: 0.9530820145735

        f1
          CV score: 0.9578320729745018 using:5
            train score: 1.0 with variance: 0.0
            test  score: 0.9578320729745018 with variance: 0.0012409641549000403
          Refitted train score: 0.971677559912854,  RMSE: 0.16774542658006547, Log-Loss:0.9718789851103684
          Refitted test  score: 0.9090909090909091,  RMSE: 0.3396831102433787, Log-Loss:3.9852434301820026
        accuracy
          CV score: 0.9589060308555399 using:5
            train score: 1.0 with variance: 0.0
            test  score: 0.9589060308555399 with variance: 0.0011767731706693366
          Refitted train score: 0.9718614718614719,  RMSE: 0.16774542658006547, Log-Loss:0.9718789851103684
          Refitted test  score: 0.8846153846153846,  RMSE: 0.3396831102433787, Log-Loss:3.9852434301820026
      ncomponents: 10
        f1
          CV score: 0.9738551506972559 using:10
            train score: 1.0 with variance: 0.0
            test  score: 0.9738551506972559 with variance: 0.0

        f1
          CV score: 0.9651245976245978 using:15
            train score: 1.0 with variance: 0.0
            test  score: 0.9651245976245978 with variance: 0.00042154452305134875
          Refitted train score: 0.9846153846153847,  RMSE: 0.12309149097933274, Log-Loss:0.523314793862284
          Refitted test  score: 0.972972972972973,  RMSE: 0.19611613513818404, Log-Loss:1.32844523047465
        accuracy
          CV score: 0.9654511453950443 using:15
            train score: 1.0 with variance: 0.0
            test  score: 0.9654511453950443 with variance: 0.0004341586087620951
          Refitted train score: 0.9848484848484849,  RMSE: 0.12309149097933274, Log-Loss:0.523314793862284
          Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.32844523047465
      ncomponents: 25
        f1
          CV score: 0.969352020976735 using:25
            train score: 1.0 with variance: 0.0
            test  score: 0.969352020976735 with variance: 0.0005

        f1
          CV score: 0.9562552900496941 using:100
            train score: 1.0 with variance: 0.0
            test  score: 0.9562552900496941 with variance: 0.00031661927496860173
          Refitted train score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
        accuracy
          CV score: 0.9567788686302011 using:100
            train score: 1.0 with variance: 0.0
            test  score: 0.9567788686302011 with variance: 0.0003202529396298101
          Refitted train score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
    random state: 1250
      ncomponents: 2
        f1
          CV score: 0.933711276683413 using:2
            train score: 0.996121799331301 with variance: 1.4221481496921525e-05
            test  score: 0.933711276683413 with variance: 0.0009631170667449793
          Refitted train score: 0

        f1
          CV score: 0.9515278920210744 using:5
            train score: 1.0 with variance: 0.0
            test  score: 0.9515278920210744 with variance: 0.0012048753527778805
          Refitted train score: 0.9664429530201342,  RMSE: 0.18176811485266747, Log-Loss:1.1411560447431308
          Refitted test  score: 0.9767441860465117,  RMSE: 0.19611613513818404, Log-Loss:1.328414476727335
        accuracy
          CV score: 0.9516483516483518 using:5
            train score: 1.0 with variance: 0.0
            test  score: 0.9516483516483518 with variance: 0.0012365656321700268
          Refitted train score: 0.9669603524229075,  RMSE: 0.18176811485266747, Log-Loss:1.1411560447431308
          Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.328414476727335
      ncomponents: 10
        f1
          CV score: 0.9597941010300561 using:10
            train score: 1.0 with variance: 0.0
            test  score: 0.9597941010300561 with variance: 0.

...

        f1
          CV score: 0.9365145602653528 using:2
            train score: 0.995615117851055 with variance: 2.9885900972501117e-05
            test  score: 0.9365145602653528 with variance: 0.000719892121601718
          Refitted train score: 0.9528535980148882,  RMSE: 0.2168633278595001, Log-Loss:1.6243662076241956
          Refitted test  score: 0.9318181818181819,  RMSE: 0.3038218101251, Log-Loss:3.188194744145602
        accuracy
          CV score: 0.9380246913580248 using:2
            train score: 0.9956656346749225 with variance: 2.9138590420688493e-05
            test  score: 0.9380246913580248 with variance: 0.000628196921201036
          Refitted train score: 0.9529702970297029,  RMSE: 0.2168633278595001, Log-Loss:1.6243662076241956
          Refitted test  score: 0.9076923076923077,  RMSE: 0.3038218101251, Log-Loss:3.188194744145602
      ncomponents: 3
        f1
          CV score: 0.9438512531137206 using:3
            train score: 1.0 with variance: 0.0
        

        f1
          CV score: 0.9455418338993515 using:5
            train score: 1.0 with variance: 0.0
            test  score: 0.9455418338993515 with variance: 0.0004906747715520582
          Refitted train score: 0.9772151898734178,  RMSE: 0.15, Log-Loss:0.7771264668726423
          Refitted test  score: 0.9690721649484536,  RMSE: 0.21483446221182986, Log-Loss:1.5941096735717277
        accuracy
          CV score: 0.945 using:5
            train score: 1.0 with variance: 0.0
            test  score: 0.945 with variance: 0.0005374999999999998
          Refitted train score: 0.9775,  RMSE: 0.15, Log-Loss:0.7771264668726423
          Refitted test  score: 0.9538461538461539,  RMSE: 0.21483446221182986, Log-Loss:1.5941096735717277
      ncomponents: 10
        f1
          CV score: 0.9624389867160952 using:10
            train score: 1.0 with variance: 0.0
            test  score: 0.9624389867160952 with variance: 0.0002289196614408475
          Refitted train score: 0.977215189873

        f1
          CV score: 0.9656611148712022 using:25
            train score: 1.0 with variance: 0.0
            test  score: 0.9656611148712022 with variance: 0.0005143316572917024
          Refitted train score: 0.9925558312655086,  RMSE: 0.08596023825918792, Log-Loss:0.25521263345993206
          Refitted test  score: 0.9565217391304348,  RMSE: 0.2480694691784169, Log-Loss:2.1254877657615876
        accuracy
          CV score: 0.9656127672387835 using:25
            train score: 1.0 with variance: 0.0
            test  score: 0.9656127672387835 with variance: 0.0005589388959565915
          Refitted train score: 0.9926108374384236,  RMSE: 0.08596023825918792, Log-Loss:0.25521263345993206
          Refitted test  score: 0.9384615384615385,  RMSE: 0.2480694691784169, Log-Loss:2.1254877657615876
      ncomponents: 50
        f1
          CV score: 0.9700866417533083 using:50
            train score: 1.0 with variance: 0.0
            test  score: 0.9700866417533083 with variance

        f1
          CV score: 0.9656705306325559 using:100
            train score: 1.0 with variance: 0.0
            test  score: 0.9656705306325559 with variance: 0.0006012143629605479
          Refitted train score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
          Refitted test  score: 0.9811320754716981,  RMSE: 0.17541160386140583, Log-Loss:1.062743882880794
        accuracy
          CV score: 0.9668614086335605 using:100
            train score: 1.0 with variance: 0.0
            test  score: 0.9668614086335605 with variance: 0.0005568613148759335
          Refitted train score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
          Refitted test  score: 0.9692307692307692,  RMSE: 0.17541160386140583, Log-Loss:1.062743882880794
    random state: 1850
      ncomponents: 2
        f1
          CV score: 0.926235094990884 using:2
            train score: 0.9926566955380833 with variance: 3.9851534921464134e-05
            test  score: 0.926235094990884 with variance: 0

        f1
          CV score: 0.9390087775188517 using:3
            train score: 0.9986927964970365 with variance: 2.563217120211298e-06
            test  score: 0.9390087775188517 with variance: 0.0006445909789545384
          Refitted train score: 0.9506493506493506,  RMSE: 0.22243913025065232, Log-Loss:1.7089706965562634
          Refitted test  score: 0.9911504424778761,  RMSE: 0.12403473458920845, Log-Loss:0.5313657906909345
        accuracy
          CV score: 0.9400546821599451 using:3
            train score: 0.9986991835526038 with variance: 2.538196329359937e-06
            test  score: 0.9400546821599451 with variance: 0.0005203588726990537
          Refitted train score: 0.9505208333333334,  RMSE: 0.22243913025065232, Log-Loss:1.7089706965562634
          Refitted test  score: 0.9846153846153847,  RMSE: 0.12403473458920845, Log-Loss:0.5313657906909345
      ncomponents: 5
        f1
          CV score: 0.947756412592194 using:5
            train score: 1.0 with variance: 

In [ ]:
results.tail()

In [11]:
#logisticregression
C = [1.0, 0.001, 1.5]

for testsize in testsizes:
  print(f"test size: {testsize}")
    
  for randomstate in randomstates:
    print(tab * 2 + f"random state: {randomstate}")
        
    for ncomponents in ncomponentss:
        print(tab * 3 + f"ncomponents: {ncomponents}")
            
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testsize, random_state = randomstate)
                     
        smote = SMOTE(random_state = randomstate)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        robustscaler = RobustScaler(quantile_range = (1, 99))
        robustscaler.fit(X_train)
                                    
        X_train = robustscaler.transform(X_train)
        X_test  = robustscaler.transform(X_test)

        pca = PCA(n_components = ncomponents)        
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_test  = pca.transform(X_test)

        best_avg_scores = {score : [None] for score in scores}
        
        # Run Grid search for each classifier
        for c in C:            
            
            lr = LogisticRegression(C = c, solver = 'liblinear', max_iter = 200, class_weight = 'balanced', random_state = randomstate)
            cv_results = cross_validate(lr, X_train, y_train, cv = cv, scoring = scores, return_train_score = True, n_jobs = jobs)

            for score in scores:
                avg_score_test = np.mean(cv_results['test_' + score])
                var_score_test = np.var(cv_results['test_' + score])
                avg_score_train = np.mean(cv_results['train_' + score])
                var_score_train = np.var(cv_results['train_' + score])

                if(best_avg_scores[score][0] is None or avg_score_test > best_avg_scores[score][0]):
                    best_avg_scores[score] = [avg_score_test, var_score_test, avg_score_train, var_score_train, c, ncomponents]

            
            for score in scores: 
            
                print(tab * 4 + str(score))
                print(tab * 5 + f"CV score: {best_avg_scores[score][0]} using:" + ','.join([str(p) for p in best_avg_scores[score][4:]]))
                print(tab * 6 + f"train score: {best_avg_scores[score][2]} with variance: {best_avg_scores[score][3]}")
                print(tab * 6 + f"test  score: {best_avg_scores[score][0]} with variance: {best_avg_scores[score][1]}")

                lr = LogisticRegression(C = best_avg_scores[score][4], solver = 'liblinear', max_iter = 200, class_weight = 'balanced', random_state = randomstate)
                    
                lr.fit(X_train, y_train)            
                y_train_pred, y_test_pred = lr.predict(X_train), lr.predict(X_test)                          
                rmse_train, rmse_test = math.sqrt(mean_squared_error(y_train, y_train_pred)), math.sqrt(mean_squared_error(y_test, y_test_pred))                    
                log_loss_train, log_loss_test = log_loss(y_train, y_train_pred), log_loss(y_test, y_test_pred)        

                score_train, score_test = get_scorer(score)(lr, X_train, y_train), get_scorer(score)(lr, X_test, y_test)

                print(tab * 5 + f"Refitted train score: {score_train},  RMSE: {rmse_train}, Log-Loss:{log_loss_train}")
                print(tab * 5 + f"Refitted test  score: {score_test},  RMSE: {rmse_test}, Log-Loss:{log_loss_test}") 

                n = len(results)
                results.at[n, 'score'] = score
                results.at[n, 'test score'] = best_avg_scores[score][0]
                results.at[n, 'train score'] = best_avg_scores[score][2]
                results.at[n, 'test variance'] = best_avg_scores[score][1]
                results.at[n, 'train variance'] = best_avg_scores[score][3]
                results.at[n, 'test rmse'] = rmse_test
                results.at[n, 'train rmse'] = rmse_train
                results.at[n, 'test log_loss'] = log_loss_test
                results.at[n, 'train log_loss'] = log_loss_train
                results.at[n, 'test size'] = testsize
                results.at[n, 'random state'] = randomstate
                results.at[n, 'estimator'] = "Smote/pca/LogisticRegression"
                results.at[n, 'estimator params'] = ','.join([str(p) for p in best_avg_scores[score][4:]])

test size: 0.08
    random state: 250
      ncomponents: 2
        f1
          CV score: 0.9489213670251899 using:1.0,2
            train score: 0.9494998259759744 with variance: 4.04793329733307e-05
            test  score: 0.9489213670251899 with variance: 0.0004620864930584774
          Refitted train score: 0.9490022172949001,  RMSE: 0.22409449036367163, Log-Loss:1.7344940040663481
          Refitted test  score: 0.975609756097561,  RMSE: 0.19611613513818404, Log-Loss:1.3284452304746504
        accuracy
          CV score: 0.9497611084567608 using:1.0,2
            train score: 0.9503238486621701 with variance: 3.721663701949046e-05
            test  score: 0.9497611084567608 with variance: 0.0004127676058958381
          Refitted train score: 0.9497816593886463,  RMSE: 0.22409449036367163, Log-Loss:1.7344940040663481
          Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.3284452304746504
        f1
          CV score: 0.9489213670251899 using:1

        f1
          CV score: 0.9773696672263119 using:1.0,15
            train score: 0.9821908416358884 with variance: 3.677518581622341e-05
            test  score: 0.9773696672263119 with variance: 0.0003712374855616511
          Refitted train score: 0.9799554565701559,  RMSE: 0.14018079405479933, Log-Loss:0.6787095798126561
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
        accuracy
          CV score: 0.9781175346392738 using:1.0,15
            train score: 0.982530039755215 with variance: 3.461762575595098e-05
            test  score: 0.9781175346392738 with variance: 0.00033813711164715595
          Refitted train score: 0.980349344978166,  RMSE: 0.14018079405479933, Log-Loss:0.6787095798126561
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
        f1
          CV score: 0.9773696672263119 using:1.0,15
            train score: 0.9821908416358884 with variance: 3.677518581622341e-05
            test  score: 

        f1
          CV score: 0.9580302851270593 using:1.0,2
            train score: 0.9595413837870568 with variance: 4.73820127257015e-05
            test  score: 0.9580302851270593 with variance: 0.0006723978505941958
          Refitted train score: 0.9606986899563319,  RMSE: 0.19738550848793068, Log-Loss:1.3456787279013074
          Refitted test  score: 0.9411764705882353,  RMSE: 0.2773500981126146, Log-Loss:2.656828953454669
        accuracy
          CV score: 0.9589527816736793 using:1.0,2
            train score: 0.9599619131326447 with variance: 4.477067063365416e-05
            test  score: 0.9589527816736793 with variance: 0.0006175490200578277
          Refitted train score: 0.961038961038961,  RMSE: 0.19738550848793068, Log-Loss:1.3456787279013074
          Refitted test  score: 0.9230769230769231,  RMSE: 0.2773500981126146, Log-Loss:2.656828953454669
        f1
          CV score: 0.9580302851270593 using:1.0,2
            train score: 0.9595413837870568 with variance:

        f1
          CV score: 0.9731264076425367 using:1.0,15
            train score: 0.9879657722955049 with variance: 8.086918166390049e-06
            test  score: 0.9731264076425367 with variance: 0.0004523686972064828
          Refitted train score: 0.9890590809628008,  RMSE: 0.10403129732205987, Log-Loss:0.37379628133020315
          Refitted test  score: 0.9142857142857143,  RMSE: 0.3396831102433787, Log-Loss:3.9852741839293175
        accuracy
          CV score: 0.9740766713417486 using:1.0,15
            train score: 0.9880963890719988 with variance: 7.5875115164253745e-06
            test  score: 0.9740766713417486 with variance: 0.0003962366793499304
          Refitted train score: 0.9891774891774892,  RMSE: 0.10403129732205987, Log-Loss:0.37379628133020315
          Refitted test  score: 0.8846153846153846,  RMSE: 0.3396831102433787, Log-Loss:3.9852741839293175
        f1
          CV score: 0.9731264076425367 using:1.0,15
            train score: 0.9879657722955049 with

        f1
          CV score: 0.938775528907702 using:1.0,2
            train score: 0.9379840429135248 with variance: 6.54275087366067e-05
            test  score: 0.938775528907702 with variance: 0.00016975067303569326
          Refitted train score: 0.9370932754880694,  RMSE: 0.2505405411716091, Log-Loss:2.168042661940331
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
        accuracy
          CV score: 0.9393408134642357 using:1.0,2
            train score: 0.9383051344026953 with variance: 6.63282894773163e-05
            test  score: 0.9393408134642357 with variance: 0.0001277178147719037
          Refitted train score: 0.9372294372294372,  RMSE: 0.2505405411716091, Log-Loss:2.168042661940331
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
        f1
          CV score: 0.938775528907702 using:1.0,2
            train score: 0.9379840429135248 with variance: 6.54275087366067e-05
            test  score: 0.938775528

...

        f1
          CV score: 0.9660951303585286 using:1.0,15
            train score: 0.9766189006958156 with variance: 3.9908485967925025e-05
            test  score: 0.9660951303585286 with variance: 0.0007230377605135606
          Refitted train score: 0.9766233766233766,  RMSE: 0.15152288168283162, Log-Loss:0.7929841508970068
          Refitted test  score: 0.9904761904761905,  RMSE: 0.12403473458920845, Log-Loss:0.5313657906909346
        accuracy
          CV score: 0.9668614086335605 using:1.0,15
            train score: 0.9770415742455384 with variance: 3.8204204400585306e-05
            test  score: 0.9668614086335605 with variance: 0.0006883537540606801
          Refitted train score: 0.9770408163265306,  RMSE: 0.15152288168283162, Log-Loss:0.7929841508970068
          Refitted test  score: 0.9846153846153847,  RMSE: 0.12403473458920845, Log-Loss:0.5313657906909346
        f1
          CV score: 0.9660951303585286 using:1.0,15
            train score: 0.9766189006958156 wit

        f1
          CV score: 0.9453397316437002 using:1.0,2
            train score: 0.9464611189764565 with variance: 2.889949727413883e-05
            test  score: 0.9453397316437002 with variance: 0.0004291647389163624
          Refitted train score: 0.9484029484029484,  RMSE: 0.22631728213974722, Log-Loss:1.7690768309024303
          Refitted test  score: 0.9523809523809523,  RMSE: 0.2480694691784169, Log-Loss:2.1254631627637353
        accuracy
          CV score: 0.9463414634146341 using:1.0,2
            train score: 0.9469512195121951 with variance: 2.8256989886972093e-05
            test  score: 0.9463414634146341 with variance: 0.0003926234384295056
          Refitted train score: 0.948780487804878,  RMSE: 0.22631728213974722, Log-Loss:1.7690768309024303
          Refitted test  score: 0.9384615384615385,  RMSE: 0.2480694691784169, Log-Loss:2.1254631627637353
        f1
          CV score: 0.9453397316437002 using:1.0,2
            train score: 0.9464611189764565 with varia

        f1
          CV score: 0.9773409907798094 using:1.0,15
            train score: 0.9876427840656348 with variance: 1.1643726569777301e-05
            test  score: 0.9773409907798094 with variance: 0.00021888309533049465
          Refitted train score: 0.9876543209876543,  RMSE: 0.11043152607484655, Log-Loss:0.4212045901818384
          Refitted test  score: 0.9662921348314608,  RMSE: 0.21483446221182986, Log-Loss:1.594121975070654
        accuracy
          CV score: 0.978048780487805 using:1.0,15
            train score: 0.9878048780487804 with variance: 1.1154074955383719e-05
            test  score: 0.978048780487805 with variance: 0.00020226055919095814
          Refitted train score: 0.9878048780487805,  RMSE: 0.11043152607484655, Log-Loss:0.4212045901818384
          Refitted test  score: 0.9538461538461539,  RMSE: 0.21483446221182986, Log-Loss:1.594121975070654
        f1
          CV score: 0.9773409907798094 using:1.0,15
            train score: 0.9876427840656348 with 

        f1
          CV score: 0.9340923772848655 using:1.0,2
            train score: 0.9342971280937027 with variance: 3.423324056587348e-05
            test  score: 0.9340923772848655 with variance: 0.0006872772946334778
          Refitted train score: 0.9315789473684212,  RMSE: 0.2602082499332666, Log-Loss:2.3385858902067973
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626415e-16
        accuracy
          CV score: 0.934894053315106 using:1.0,2
            train score: 0.934891492871949 with variance: 3.445447392835741e-05
            test  score: 0.934894053315106 with variance: 0.0006073018250591373
          Refitted train score: 0.9322916666666666,  RMSE: 0.2602082499332666, Log-Loss:2.3385858902067973
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626415e-16
        f1
          CV score: 0.9340923772848655 using:1.0,2
            train score: 0.9342971280937027 with variance: 3.423324056587348e-05
            test  score: 0.93409

        f1
          CV score: 0.9686133853151396 using:1.0,15
            train score: 0.981477772138606 with variance: 2.0677355781346142e-05
            test  score: 0.9686133853151396 with variance: 0.0005683030313009683
          Refitted train score: 0.9815303430079155,  RMSE: 0.13501543121683043, Log-Loss:0.6296151936505348
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626415e-16
        accuracy
          CV score: 0.9688311688311689 using:1.0,15
            train score: 0.9817737636955878 with variance: 1.938859162326203e-05
            test  score: 0.9688311688311689 with variance: 0.0005801990217574637
          Refitted train score: 0.9817708333333334,  RMSE: 0.13501543121683043, Log-Loss:0.6296151936505348
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626415e-16
        f1
          CV score: 0.9686133853151396 using:1.0,15
            train score: 0.981477772138606 with variance: 2.0677355781346142e-05
            test  score:

In [12]:
results.tail()

,score,test score,train score,test variance,train variance,test rmse,train rmse,test log_loss,train log_loss,test size,random state,estimator,estimator params
2235,accuracy,0.968831,0.999349,0.000580199,1.69763e-06,0.124035,0,0.531366,9.99201e-16,0.2,2050,Smote/pca/LogisticRegression,"1.0,100"
2236,f1,0.968613,0.999349,0.000568303,1.69763e-06,0.124035,0,0.531366,9.99201e-16,0.2,2050,Smote/pca/LogisticRegression,"1.0,100"
2237,accuracy,0.968831,0.999349,0.000580199,1.69763e-06,0.124035,0,0.531366,9.99201e-16,0.2,2050,Smote/pca/LogisticRegression,"1.0,100"
2238,f1,0.968613,0.999349,0.000568303,1.69763e-06,0.124035,0,0.531366,9.99201e-16,0.2,2050,Smote/pca/LogisticRegression,"1.0,100"
2239,accuracy,0.968831,0.999349,0.000580199,1.69763e-06,0.124035,0,0.531366,9.99201e-16,0.2,2050,Smote/pca/LogisticRegression,"1.0,100"


In [13]:
#svc
kernels = ['linear', 'rbf']
C = [0.001, 1, 1.5]

for testsize in testsizes:
  print(f"test size: {testsize}")
    
  for randomstate in randomstates:
    print(tab * 2 + f"random state: {randomstate}")
        
    for ncomponents in ncomponentss:
        print(tab * 3 + f"ncomponents: {ncomponents}")

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testsize, random_state = randomstate)
                     
        smote = SMOTE(random_state = randomstate)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        robustscaler = RobustScaler(quantile_range = (1, 99))
        robustscaler.fit(X_train)
                                    
        X_train = robustscaler.transform(X_train)
        X_test  = robustscaler.transform(X_test)

        pca = PCA(n_components = ncomponents)        
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_test  = pca.transform(X_test)

        best_avg_scores = {score : [None] for score in scores}
        
        # Run Grid search for each classifier
        for kernel in kernels:
            for c in C:
                    
                svc = SVC(kernel = kernel, C = c, max_iter = 200, class_weight = 'balanced', random_state = randomstate)
                cv_results = cross_validate(svc, X_train, y_train, cv = cv, scoring = scores, return_train_score = True, n_jobs = jobs)

                for score in scores:
                    avg_score_test = np.mean(cv_results['test_' + score])
                    var_score_test = np.var(cv_results['test_' + score])
                    avg_score_train = np.mean(cv_results['train_' + score])
                    var_score_train = np.var(cv_results['train_' + score])

                    if(best_avg_scores[score][0] is None or avg_score_test > best_avg_scores[score][0]):
                        best_avg_scores[score] = [avg_score_test, var_score_test, avg_score_train, var_score_train, kernel, c, ncomponents]


        for score in scores: 
            
            print(tab * 4 + str(score))
            print(tab * 5 + f"CV score: {best_avg_scores[score][0]} using:" + ','.join([str(p) for p in best_avg_scores[score][4:]]))
            print(tab * 6 + f"train score: {best_avg_scores[score][2]} with variance: {best_avg_scores[score][3]}")
            print(tab * 6 + f"test  score: {best_avg_scores[score][0]} with variance: {best_avg_scores[score][1]}")

            svc = SVC(kernel = best_avg_scores[score][4], C = best_avg_scores[score][5], max_iter = 200, class_weight = 'balanced', random_state = randomstate)
            
            svc.fit(X_train, y_train)            
            y_train_pred, y_test_pred = svc.predict(X_train), svc.predict(X_test)                          
            rmse_train, rmse_test = math.sqrt(mean_squared_error(y_train, y_train_pred)), math.sqrt(mean_squared_error(y_test, y_test_pred))                    
            log_loss_train, log_loss_test = log_loss(y_train, y_train_pred), log_loss(y_test, y_test_pred)        

            score_train, score_test = get_scorer(score)(svc, X_train, y_train), get_scorer(score)(svc, X_test, y_test)
            
            print(tab * 5 + f"Refitted train score: {score_train},  RMSE: {rmse_train}, Log-Loss:{log_loss_train}")
            print(tab * 5 + f"Refitted test  score: {score_test},  RMSE: {rmse_test}, Log-Loss:{log_loss_test}")            
            
            n = len(results)
            results.at[n, 'score'] = score
            results.at[n, 'test score'] = best_avg_scores[score][0]
            results.at[n, 'train score'] = best_avg_scores[score][2]
            results.at[n, 'test variance'] = best_avg_scores[score][1]
            results.at[n, 'train variance'] = best_avg_scores[score][3]
            results.at[n, 'test rmse'] = rmse_test
            results.at[n, 'train rmse'] = rmse_train
            results.at[n, 'test log_loss'] = log_loss_test
            results.at[n, 'train log_loss'] = log_loss_train
            results.at[n, 'test size'] = testsize
            results.at[n, 'random state'] = randomstate
            results.at[n, 'estimator'] = "Smote/pca/SVC"
            results.at[n, 'estimator params'] = ','.join([str(p) for p in best_avg_scores[score][4:]])

test size: 0.08
    random state: 250
      ncomponents: 2
        f1
          CV score: 0.9523987869225913 using:rbf,1.5,2
            train score: 0.9559961575115761 with variance: 4.853054281615784e-05
            test  score: 0.9523987869225913 with variance: 0.00041240037273207534
          Refitted train score: 0.9571106094808127,  RMSE: 0.20367797170338584, Log-Loss:1.4328348268518858
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
        accuracy
          CV score: 0.9541328236980411 using:rbf,1.5,2
            train score: 0.9574202290019505 with variance: 4.365968240347512e-05
            test  score: 0.9541328236980411 with variance: 0.0003577152609122519
          Refitted train score: 0.9585152838427947,  RMSE: 0.20367797170338584, Log-Loss:1.4328348268518858
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
      ncomponents: 3
        f1
          CV score: 0.9609236066002065 using:rbf,1,3
            train

        f1
          CV score: 0.9645816673586658 using:rbf,1.5,5
            train score: 0.9702345300434047 with variance: 3.0233397432373205e-05
            test  score: 0.9645816673586658 with variance: 0.0005586042571623029
          Refitted train score: 0.9691629955947136,  RMSE: 0.17407765595569785, Log-Loss:1.0466347799156719
          Refitted test  score: 0.9090909090909091,  RMSE: 0.3396831102433787, Log-Loss:3.9852434301820026
        accuracy
          CV score: 0.9654043945769051 using:rbf,1.5,5
            train score: 0.9707800483410238 with variance: 2.75159500116759e-05
            test  score: 0.9654043945769051 with variance: 0.0005264548651340538
          Refitted train score: 0.9696969696969697,  RMSE: 0.17407765595569785, Log-Loss:1.0466347799156719
          Refitted test  score: 0.8846153846153846,  RMSE: 0.3396831102433787, Log-Loss:3.9852434301820026
      ncomponents: 10
        f1
          CV score: 0.9777734584693629 using:linear,1.5,10
            trai

        f1
          CV score: 0.9758559464059168 using:linear,1.5,15
            train score: 0.984267911574606 with variance: 0.00010381007987555907
            test  score: 0.9758559464059168 with variance: 0.00026296809516800844
          Refitted train score: 0.9869565217391305,  RMSE: 0.11396057645963795, Log-Loss:0.4485589990569804
          Refitted test  score: 0.972972972972973,  RMSE: 0.19611613513818404, Log-Loss:1.32844523047465
        accuracy
          CV score: 0.9762505843852267 using:linear,1,15
            train score: 0.9886339998535121 with variance: 1.0013784598839482e-05
            test  score: 0.9762505843852267 with variance: 0.0002503616686129777
          Refitted train score: 0.987012987012987,  RMSE: 0.11396057645963795, Log-Loss:0.4485555375962436
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
      ncomponents: 25
        f1
          CV score: 0.9821717358796012 using:linear,1,25
            train score: 0.9940053685358

        f1
          CV score: 0.9824165206773902 using:linear,1.5,50
            train score: 0.9983710380582066 with variance: 4.722704769785739e-06
            test  score: 0.9824165206773902 with variance: 0.00012562137045542748
          Refitted train score: 0.9956521739130434,  RMSE: 0.0657951694959769, Log-Loss:0.14951851253208184
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
        accuracy
          CV score: 0.9827255726975223 using:linear,1.5,50
            train score: 0.9983783783783784 with variance: 4.67494521548566e-06
            test  score: 0.9827255726975223 with variance: 0.00012035439699203582
          Refitted train score: 0.9956709956709957,  RMSE: 0.0657951694959769, Log-Loss:0.14951851253208184
          Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
      ncomponents: 100
        f1
          CV score: 0.9777745559985502 using:linear,1,100
            train score: 1.0 with variance: 0.0
            t

        f1
          CV score: 0.9485448709174866 using:rbf,1.5,2
            train score: 0.9481392797181535 with variance: 4.446025906063489e-05
            test  score: 0.9485448709174866 with variance: 0.000328602577518116
          Refitted train score: 0.950892857142857,  RMSE: 0.22013211891892487, Log-Loss:1.6736992895759402
          Refitted test  score: 0.9767441860465117,  RMSE: 0.19611613513818404, Log-Loss:1.328414476727335
        accuracy
          CV score: 0.9494017094017094 using:rbf,1.5,2
            train score: 0.9487936306118124 with variance: 4.3700617506976506e-05
            test  score: 0.9494017094017094 with variance: 0.00031602075851160034
          Refitted train score: 0.9515418502202643,  RMSE: 0.22013211891892487, Log-Loss:1.6736992895759402
          Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.328414476727335
      ncomponents: 3
        f1
          CV score: 0.9640347641509981 using:rbf,1,3
            train score

...

        f1
          CV score: 0.9776489657301637 using:linear,1.5,10
            train score: 0.9837809824302559 with variance: 1.684062355959514e-05
            test  score: 0.9776489657301637 with variance: 0.00038975378322297455
          Refitted train score: 0.9850746268656716,  RMSE: 0.12186666955535813, Log-Loss:0.5129560830800121
          Refitted test  score: 0.9574468085106385,  RMSE: 0.2480694691784169, Log-Loss:2.125487765761587
        accuracy
          CV score: 0.9777160493827161 using:linear,1.5,10
            train score: 0.9839104842716815 with variance: 1.6881908882945423e-05
            test  score: 0.9777160493827161 with variance: 0.00039050449626581345
          Refitted train score: 0.9851485148514851,  RMSE: 0.12186666955535813, Log-Loss:0.5129560830800121
          Refitted test  score: 0.9384615384615385,  RMSE: 0.2480694691784169, Log-Loss:2.125487765761587
      ncomponents: 15
        f1
          CV score: 0.9802354718017368 using:linear,1,15
         

        f1
          CV score: 0.9872768581629341 using:linear,1,25
            train score: 0.9943376137547666 with variance: 5.602728248803051e-06
            test  score: 0.9872768581629341 with variance: 6.575254033131848e-05
          Refitted train score: 0.9949748743718593,  RMSE: 0.07071067811865475, Log-Loss:0.1726938819745544
          Refitted test  score: 0.9690721649484536,  RMSE: 0.21483446221182986, Log-Loss:1.5941096735717275
        accuracy
          CV score: 0.9875 using:linear,1,25
            train score: 0.994375 with variance: 5.468749999999933e-06
            test  score: 0.9875 with variance: 6.250000000000011e-05
          Refitted train score: 0.995,  RMSE: 0.07071067811865475, Log-Loss:0.1726938819745544
          Refitted test  score: 0.9538461538461539,  RMSE: 0.21483446221182986, Log-Loss:1.5941096735717275
      ncomponents: 50
        f1
          CV score: 0.9798694507555267 using:linear,1,50
            train score: 0.9968652037617556 with variance: 

        f1
          CV score: 0.9851211698474227 using:linear,1,100
            train score: 1.0 with variance: 0.0
            test  score: 0.9851211698474227 with variance: 8.512651732091637e-05
          Refitted train score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
          Refitted test  score: 0.967741935483871,  RMSE: 0.21483446221182986, Log-Loss:1.5941219750706541
        accuracy
          CV score: 0.9852152965974103 using:linear,1,100
            train score: 1.0 with variance: 0.0
            test  score: 0.9852152965974103 with variance: 8.550516767504983e-05
          Refitted train score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
          Refitted test  score: 0.9538461538461539,  RMSE: 0.21483446221182986, Log-Loss:1.5941219750706541
    random state: 1250
      ncomponents: 2
        f1
          CV score: 0.9433527003147256 using:linear,1,2
            train score: 0.9413540472569686 with variance: 0.00029991826824178874
            test  score: 0.943

        f1
          CV score: 0.9628188537626288 using:linear,1,3
            train score: 0.9666738643532454 with variance: 3.9296799788045224e-05
            test  score: 0.9628188537626288 with variance: 0.000299158447011999
          Refitted train score: 0.9516908212560385,  RMSE: 0.2208630521496931, Log-Loss:1.6848417635789668
          Refitted test  score: 0.9545454545454546,  RMSE: 0.2480694691784169, Log-Loss:2.125487765761587
        accuracy
          CV score: 0.9634146341463413 using:linear,1,3
            train score: 0.9670731707317073 with variance: 3.8667459845330225e-05
            test  score: 0.9634146341463413 with variance: 0.00029744199881023256
          Refitted train score: 0.9512195121951219,  RMSE: 0.2208630521496931, Log-Loss:1.6848417635789668
          Refitted test  score: 0.9384615384615385,  RMSE: 0.2480694691784169, Log-Loss:2.125487765761587
      ncomponents: 5
        f1
          CV score: 0.9694871794871794 using:rbf,1,5
            train score

        f1
          CV score: 0.9596476968269366 using:rbf,1.5,5
            train score: 0.9648522009515874 with variance: 1.2949996292395139e-05
            test  score: 0.9596476968269366 with variance: 0.0002679101532337484
          Refitted train score: 0.962962962962963,  RMSE: 0.19094065395649334, Log-Loss:1.2592345518710173
          Refitted test  score: 0.9911504424778761,  RMSE: 0.12403473458920845, Log-Loss:0.5313657906909345
        accuracy
          CV score: 0.9609706083390293 using:rbf,1.5,5
            train score: 0.9654955793392276 with variance: 1.0976135512508002e-05
            test  score: 0.9609706083390293 with variance: 0.00020063830115274548
          Refitted train score: 0.9635416666666666,  RMSE: 0.19094065395649334, Log-Loss:1.2592345518710173
          Refitted test  score: 0.9846153846153847,  RMSE: 0.12403473458920845, Log-Loss:0.5313657906909345
      ncomponents: 10
        f1
          CV score: 0.967062271062271 using:rbf,1,10
            train 

In [14]:
results.tail()

,score,test score,train score,test variance,train variance,test rmse,train rmse,test log_loss,train log_loss,test size,random state,estimator,estimator params
2683,accuracy,0.968831,0.973963,0.000715129,2.94347e-05,0.124035,0.161374,0.531366,0.899447,0.2,2050,Smote/pca/SVC,"rbf,1,25"
2684,f1,0.971938,0.998041,0.000443128,2.55765e-06,0.124035,0.051031,0.531378,0.0899447,0.2,2050,Smote/pca/SVC,"linear,1,50"
2685,accuracy,0.971429,0.998048,0.000499241,2.54095e-06,0.124035,0.051031,0.531378,0.0899447,0.2,2050,Smote/pca/SVC,"linear,1,50"
2686,f1,0.967062,0.978693,0.000871542,3.92429e-05,0.124035,0.144338,0.531366,0.719558,0.2,2050,Smote/pca/SVC,"rbf,1,100"
2687,accuracy,0.968831,0.97917,0.000715129,3.63238e-05,0.124035,0.144338,0.531366,0.719558,0.2,2050,Smote/pca/SVC,"rbf,1,100"


In [16]:
results.to_csv(path + "classification-results-3-pca.csv")